In [1]:
'''
Defines the jwk_to_pem function which converts jwk format pubkeys to pem format
'''
from cryptography.hazmat.primitives.asymmetric import rsa
from cryptography.hazmat.primitives import serialization
import base64

def b64_to_int(b64_string):
    """Convert base64 to integer."""
    return int.from_bytes(base64.urlsafe_b64decode(b64_string + '=='), 'big')

def jwk_to_pem(jwk):
    """Convert a JWK to PEM format."""
    if jwk["kty"] != "RSA":
        raise ValueError("Unsupported key type")
    
    # Convert modulus and exponent from base64 to integers
    modulus = b64_to_int(jwk["n"])
    exponent = b64_to_int(jwk["e"])

    # Build the RSA public key
    public_key = rsa.RSAPublicNumbers(exponent, modulus).public_key()

    # Convert the RSA public key to PEM format
    pem = public_key.public_bytes(
        encoding=serialization.Encoding.PEM,
        format=serialization.PublicFormat.SubjectPublicKeyInfo
    )

    return pem.decode('utf-8')

In [2]:
'''
Defines verifyToken and the helper functions invoked to perform the function (e.g. to pull the pubkeys from the Cognito endpoint
'''

import requests

region = 'us-east-1'
userPoolId = 'us-east-1_kKJIKYQVF'

jwksUrl = f"https://cognito-idp.{region}.amazonaws.com/{userPoolId}/.well-known/jwks.json"

def getPubKeysFromUrl(jwksUrl):
    
    print(f"Retrieving public keys from {jwksUrl}")

    response = requests.get(jwksUrl)

    if response.status_code != 200:
      raise Exception("Unable to retrieve public keys from {jwksUrl}")
    
    pubkeys_jwk_json = response.json()
    pubkeys_pem = {}

    for pubkey_jwk in pubkeys_jwk_json['keys']:
      kid = pubkey_jwk['kid']
      pubkeys_pem[kid] = {
         "pubkey": jwk_to_pem(pubkey_jwk),
         "alg": pubkey_jwk['alg']
      }
    
    return pubkeys_pem

pubkeys = getPubKeysFromUrl(jwksUrl)
print(pubkeys)

Retrieving public keys from https://cognito-idp.us-east-1.amazonaws.com/us-east-1_kKJIKYQVF/.well-known/jwks.json
{'oyeJIIULWKla+DehPcWXVDS1KeRrZOoatXvrB8X4cxE=': {'pubkey': '-----BEGIN PUBLIC KEY-----\nMIIBIjANBgkqhkiG9w0BAQEFAAOCAQ8AMIIBCgKCAQEAyQDS7IObHvzXhAEE+q9P\njXsLhN4/3FwRGf440oYIlni5/b5VIaC0wzNClC3VMdsD9wu7r+f3U7SzKXrSR38L\nEaWRT0ODWFCYaEDpdxnim5IPcuKZCvWCd2VPJVzBtn8ygLm+dVGRsrs/Kbkthq1x\naMJFNfuruO8Vfgnu31b/UK5JZrppJau4BSZael1SJG4x9/oo6pHo80MkN0XWez+M\n94qo9b8JfJjCiimDbZfsB7kssz8Zyc0rBIN/sY9ZAikcS/K+tgqtcBBs6IIoAtco\nImo+pnOapkCEYpi8yJEnRYgm07qqKGd4Vi7jR2FwdALorRGUQ8VgT+NJFb9FpNbs\nHQIDAQAB\n-----END PUBLIC KEY-----\n', 'alg': 'RS256'}, '4iAZ/0S3Dg8hioi08+HgChnOWJreTR9hS2e713hN4IU=': {'pubkey': '-----BEGIN PUBLIC KEY-----\nMIIBIjANBgkqhkiG9w0BAQEFAAOCAQ8AMIIBCgKCAQEA4K8Pj4NYtYQkWUj5yBa2\noraQfNz65UohPjgQxJBopkTEEfJMBKdhGiyAOyMvSKNfAMrTL+L0L+eeNEPOYwla\nQxjK/Bcc7mWDHf0Ys0hIek5phUWQZx7cNoOgJDxWl6VqxlgtD/q7azDYF/Kq1xoS\nfkxLKXxJMy9FsGKHHbFrr2+m5JIei9egjs7T+5J50A5pTBInzXhtP8aSLxk

In [22]:
import jwt
from jwt.exceptions import ExpiredSignatureError, InvalidTokenError

with open('auth-token', 'rb') as f:
    token = f.read().decode()

def verifyToken(token):

    kid = jwt.get_unverified_header(token).get('kid')
    if not kid:
        print('kid field required in the JWT header')

    pubkeys = getPubKeysFromUrl(jwksUrl)
    if not pubkeys.get(kid):
        print('Unauthenticated. User not in the authorized user pool')

    try:
        claims = jwt.decode(token, pubkeys[kid]['pubkey'], algorithms=[pubkeys[kid]['alg']])
        return claims

    except Exception as e: 
        if isinstance(e, ExpiredSignatureError):
            print("Signature expired. Please obtain a new auth token. ")
        elif isinstance(e, InvalidTokenError):
            print("Invalid signature")
        else:
            print(f"Exception raised: {e}")
        return None
  
print(verifyToken(token))

Retrieving public keys from https://cognito-idp.us-east-1.amazonaws.com/us-east-1_kKJIKYQVF/.well-known/jwks.json
{'sub': '8458e4b8-c071-7066-dc6c-e73ba3ba23c4', 'iss': 'https://cognito-idp.us-east-1.amazonaws.com/us-east-1_kKJIKYQVF', 'client_id': '19mnap32f93ldfsiip8d946v3p', 'origin_jti': '62706e6a-da1a-41cf-b330-2699eb85bda9', 'event_id': '43ece6bd-9bdb-47a1-941a-be7bca625494', 'token_use': 'access', 'scope': 'aws.cognito.signin.user.admin', 'auth_time': 1724925547, 'exp': 1724929147, 'iat': 1724925547, 'jti': 'a4363f2d-58b8-4949-b318-3d7b5030d0b4', 'username': '8458e4b8-c071-7066-dc6c-e73ba3ba23c4'}
